In [10]:
import argparse
from webcam_utils import realtime_emotions
from prediction_utils import prediction_path

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 


ImportError: cannot import name 'AutoGraphError' from 'tensorflow.python.autograph.impl.api' (C:\Users\gkwogus\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\autograph\impl\api.py)

In [68]:
# for running realtime emotion detection
import easydict

def run_realtime_emotion():
    realtime_emotions()

# to run emotion detection on image saved on disk
def run_detection_path(img_str):
    return(prediction_path(img_str))

def emotion_detection(img_str):
    args = easydict.EasyDict({"func_name": "emo_path", "img_str": img_str})
    
    if args.func_name == "emo_realtime":
        result=run_realtime_emotion()
    elif args.func_name == "emo_path":
        result=run_detection_path(args.img_str)
    else:
        print("Usage: python main.py <function name>")
    
    return result
    

In [ ]:
from flask import Flask, request, jsonify
app = Flask(__name__)
host = 'localhost'
port = 5000

@app.route('/emotion', methods = ['POST'])
def analysisEmotiom():
    imageData = request.get_json()['image']
    return jsonify({'result': emotion_detection(imageData)})

@app.route('/recommendations', methods = ['POST'])
def analysisEmotionAndReturnRecommends():
    imageData = request.get_json()['image']
    return jsonify({'result': 'test'})

@app.route('/wakeup', methods = ['GET'])
def wakeup():
    return jsonify({'status': 'open'}) 

app.run(host=host, port=port)